In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import json
from pathlib import Path
import numpy as np
from tensorflow.keras.preprocessing import image

# Defining constants
img_shape = (832, 535)
class_names = ['Assam_Type', 'Metal_Sheet', 'RCC', 'Vacant']
model_weights_path = '/home/storage/Backup_E/phd/CNN_Building_classification/Models/efficent_net_30000/Efficient_net_RS_30000/model.weights.h5'

# Initializing the model
def Init_model():
    global img_shape
    global class_names
    classes = len(class_names)

    model = Sequential()

    pretrained_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=(img_shape[0],img_shape[1],3),
        pooling='avg',
        classes=classes,
        classifier_activation="softmax",
    )

    for layer in pretrained_model.layers:
        layer.trainable = False

    model.add(pretrained_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(4, activation='softmax'))

    return model

# Initializing and compiling the model
model = Init_model()

# Building the model by passing an input through it
model.build((None, img_shape[0], img_shape[1], 3))

# Loading the weights
model.load_weights(model_weights_path)

# Compiling the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# Defining function to preprocess the image
def preprocess_image(img_path, target_size=(832, 535)):
    img = image.load_img(img_path, target_size=target_size)  # Load image
    img_array = image.img_to_array(img)  # Convert to NumPy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)  # Preprocess for EfficientNet
    return img_array

In [ ]:
## This is for predicting Single image
#class names
class_names = ['Assam_Type', 'Metal_Sheet', 'RCC', 'Vacant']

# Load and preprocess image
img_path = "path_to_your_image.jpg"  # Update with your image path
input_image = preprocess_image(img_path)

# Perform prediction
predictions = model.predict(input_image)

# Get the predicted class
predicted_class = np.argmax(predictions, axis=1)[0]
predicted_label = class_names[predicted_class]

# Print results
print(f"Predicted Class: {predicted_label}")
print(f"Confidence Scores: {predictions}")

In [ ]:
## Use this for multiple image predictions
import os

image_folder = "path_to_images_directory"  # Update with the folder path

for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    input_image = preprocess_image(img_path)
    predictions = model.predict(input_image)
    
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_label = class_names[predicted_class]
    
    print(f"Image: {img_name}, Predicted Class: {predicted_label}")
